# MODEL SELECTION

In [3]:
import pandas as pd
import numpy as np

In [4]:
PATH = '/home/fernanda/Documents/Projects/data_product_architecture_2021/'
df = pd.read_pickle(PATH+"clean.pkl")

In [5]:
def d_cols_inicial(df):
    df = df.drop(df.filter(regex='descrip').columns, axis=1)
    df = df.drop(df.filter(regex='comment').columns, axis=1)
    df = df.drop(df.filter(regex='name').columns, axis=1)
    df = df.drop(df.filter(regex='license').columns, axis=1)
    df = df.drop(df.filter(regex='address').columns, axis=1)
    df = df.drop(df.filter(regex='date').columns, axis=1)
    df = df.drop(df.filter(regex='latitude').columns, axis=1)
    df = df.drop(df.filter(regex='longitude').columns, axis=1)
    return df

In [6]:
def enc_fac_type(df):
    # encoding facility type
    ohe = OneHotEncoder(handle_unknown='ignore')
    fac_ty_df = pd.DataFrame(ohe.fit_transform(df[['facility_type']]).toarray())
    fac_ty_df.columns = ohe.get_feature_names(['fac_type'])
    fac_ty_df['inspection_id'] = df.inspection_id
    df_p = pd.merge(df  , fac_ty_df, how="left", on=["inspection_id"])
    df_p = df_p.drop(['facility_type'], axis = 1)
    return df_p

In [7]:
def enc_zip(df):
    # encoding zip
    ohe = OneHotEncoder(handle_unknown='ignore')
    zip_df = pd.DataFrame(ohe.fit_transform(df[['zip']]).toarray())
    zip_df.columns = ohe.get_feature_names(['zip'])
    zip_df['inspection_id'] = df.inspection_id
    df_p = pd.merge(df, zip_df, how="left", on=["inspection_id"])
    df_p = df_p.drop(['zip'], axis = 1)
    return df_p

In [8]:
def enc_risk(df):
    # encoding risk
    ohe = OneHotEncoder(handle_unknown='ignore')
    risk_df = pd.DataFrame(ohe.fit_transform(df[['risk']]).toarray())
    risk_df.columns = ohe.get_feature_names(['risk'])
    risk_df['inspection_id'] = df.inspection_id
    df_p = pd.merge(df, risk_df, how="left", on=["inspection_id"])
    df_p = df_p.drop(['risk'], axis = 1)
    return df_p

In [9]:
def enc_insp_type(df):
    # encoding risk
    ohe = OneHotEncoder(handle_unknown='ignore')
    insp_ty_df = pd.DataFrame(ohe.fit_transform(df[['inspection_type']]).toarray())
    insp_ty_df.columns = ohe.get_feature_names(['insp_type'])
    insp_ty_df['inspection_id'] = df.inspection_id
    df_p = pd.merge(df, insp_ty_df, how="left", on=["inspection_id"])
    df_p = df_p.drop(['inspection_type'], axis = 1)
    return df_p

In [10]:
def enc_label(df):
    # encoding label
    ohe = OneHotEncoder(handle_unknown='ignore')
    label_df = pd.DataFrame(ohe.fit_transform(df[['results']]).toarray())
    label_df.columns = ohe.get_feature_names(['label'])
    label_df['inspection_id'] = df.inspection_id
    label_df = label_df.drop('label_pass', axis=1)
    label_df = label_df.rename(columns={'label_fail': 'label'})
    df_p = pd.merge(df, label_df, how="left", on=["inspection_id"])
    df_p = df_p.drop(['results'], axis = 1)
    return df_p

In [11]:
df = d_cols_inicial(df)
df = enc_fac_type(df)
df = enc_zip(df)
df = enc_risk(df)
df = enc_insp_type(df)
df = enc_label(df)

In [12]:
def sep_train_test(df_p):
    # separar en train  y test 
    df_train, df_test = train_test_split(df_p ,test_size=0.25, random_state=100, stratify = df_p['label'])
    df_train['ind_train'] = 1
    df_test['ind_train'] = 0
    return df_train, df_test

In [13]:
df_train, df_test = sep_train_test(df)

<ipython-input-12-f092e44c7b12>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['ind_train'] = 1
<ipython-input-12-f092e44c7b12>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['ind_train'] = 0


In [14]:
df_final = df_train.append(df_test)

In [15]:
df_final['ind_train'].value_counts(normalize=True)

1    0.749999
0    0.250001
Name: ind_train, dtype: float64

In [18]:
df_train['ind_train'] = 1
df_test['ind_train'] = 0

<ipython-input-18-220790365634>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['ind_train'] = 1
<ipython-input-18-220790365634>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['ind_train'] = 0


In [19]:
PATH = '/home/fernanda/Documents/Projects/data_product_architecture_2021/'
df = pd.read_pickle(PATH+"clean.pkl")

In [20]:
def feature_engineering_all(df):
    df = d_cols_inicial(df)
    df = enc_fac_type(df)
    df = enc_zip(df)
    df = enc_risk(df)
    df = enc_insp_type(df)
    df = enc_label(df)
    df_train, df_test = sep_train_test(df)
    df_final = df_train.append(df_test)
    d = {'descripcion': ['registros total encoding',
                         'columnas total encoding',
                         'registros train', 
                         'registros test']
                        , 'valor': [df_final.shape[0],
                                    df_final.shape[1],
                                    df_train.shape[0],
                                    df_test.shape[0]]}
    metadata_fe = pd.DataFrame(data=d) 
    return df_final, metadata_fe

In [21]:
df_final, metadata_fe =feature_engineering_all(df) 

<ipython-input-12-f092e44c7b12>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['ind_train'] = 1
<ipython-input-12-f092e44c7b12>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['ind_train'] = 0


In [22]:
df_final

,inspection_id,ind_reinspect,violation1,violation10,violation11,violation12,violation13,violation14,violation15,violation16,...,risk_Risk 2 (Medium),risk_Risk 3 (Low),insp_type_canvass,insp_type_complaint,insp_type_consultation,insp_type_license,insp_type_suspected food poisoning,insp_type_task force,label,ind_train
93652,1557207,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
171396,455677,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
2488,67822,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
80450,1683240,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
160952,586184,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142705,1188311,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
130481,1335428,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
11571,2356204,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
96863,1537477,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [23]:
metadata_fe

,descripcion,valor
0,registros total encoding,184019
1,columnas total encoding,161
2,registros train,138014
3,registros test,46005


In [24]:
PATH = '/home/fernanda/Documents/Projects/data_product_architecture_2021/'
df_final.to_pickle(PATH+'data_fe.pkl', compression='infer', protocol=5, storage_options=None)